In [21]:
# PEA implementation of stochastic growth model adapted from Fabrice Collard's Matlab code, http://fabcol.free.fr/
# tested in Julia 1.5.0
# this code is part of chapter 5, "Advanced Numerical Techniques" from the book: "Introduction to Quantitative Macroeconomics with Julia"
# Academic Press - Elsevier
# for comments, email at: petre(dot)caraiani(at)gmail(dot)com

using DelimitedFiles;
using LinearAlgebra;
using Plots;

# set algorithm parameters
long  = 200;
init  = 50;
slong = init+long;
T     = init+1:slong-1;
T1    = init+2:slong;
tol   = 1e-6;
crit  = 1;

#set model parameters
gam     = 1;
sigma   = 1;
delta   = 0.1;
beta    = 0.95;
alpha   = 0.3;
ab      = 0;
rho     = 0.9;
se      = 0.01;

#steady state
ksy =(alpha*beta)/(1-beta*(1-delta));
yss = ksy^(alpha/(1-alpha));
kss = yss^(1/alpha);
iss = delta*kss;
css = yss-iss;
csy = css/yss;
lss = css^(-sigma);

In [22]:
#generate own shocks
#using(DataFrames)
#srand(1)
#e   = se*randn(slong,1);
#ee = DataFrame(e)
#writetable("output.csv", ee);
#load shocks from Collard(2015) generated in Matlab to find the same solution
e=readdlm("e1_noise.csv");

In [23]:
a   = zeros(slong,1);
a[1]=  ab+e[1];
for i=2:slong;
a[i]=rho*a[i-1]+(1-rho)*ab+e[i];
end

In [24]:
#initial solution
ncont  = 3;                  # of static equations
nbend  = 1;                  # endogenous predetermined variables
nshoc  = 1;                  # of shocks
nback  = nbend+nshoc;        # of state variables
nforw  = 1;                  # of costate variables
nstat  = nback+nforw;        # of state and costate variables
Mcc    = zeros(ncont,ncont);
Mcs    = zeros(ncont,nstat);
Mss0   = zeros(nstat,nstat);
Mss1   = zeros(nstat,nstat);
Msc0   = zeros(nstat,ncont);
Msc1   = zeros(nstat,ncont);
Mse    = zeros(nstat,nshoc);

In [25]:
#setup the matrices
# Output
Mcc[1,1] = 1;
Mcs[1,1] = alpha;
Mcs[1,2] = 1;

# investment
Mcc[2,1] = 1;
Mcc[2,2] = -iss/yss;
Mcc[2,3] = -css/yss;

# consumption
Mcc[3,3] = -sigma;
Mcs[3,3] = 1;

# capital
Mss0[1,1] = 1;
Mss1[1,1] = delta-1;
Msc1[1,2] = delta;

# technology shock
Mss0[2,2] = 1;
Mss1[2,2] = -rho;
Mse[2,1]  = 1;

# Euler
Mss0[3,1] = (1-beta*(1-delta));
Mss0[3,3] = -1;
Mss1[3,3] = 1;
Msc0[3,1] = (1-beta*(1-delta));

In [26]:
# Solving the system   
M0=inv(Mss0-Msc0*inv(Mcc)*Mcs);
M1=(Mss1-Msc1*inv(Mcc)*Mcs);
W=-M0*M1;

# MU -> eigenvalues, P -> eigenvectors
v,w = eigen(W,sortby = x -> -abs(x));
r=[abs.(v) w']
vsize=size(v,1)
for i = 1:vsize
    for j = i+1:vsize
        if  real(r[i,1])> real(r[j,1])
            tmp = r[i,:];
            r[i,:]=r[j,:];
            r[j,:]=tmp;
        elseif real(r[i,1])== real(r[j,1])
            if imag(r[i,1])> imag(r[j,1])
            tmp = r[i,:];
            r[i,:]=r[j,:];
            r[j,:]=tmp;   
            end;    
        end;
    end;     
end;

lam= r[:,1];
P=r[:,2:4]';

Q=inv(P);
lam=diagm(lam);

In [27]:
#Direct solution
Gamma=-inv(Q[nback+1:nstat,nback+1:nstat])*Q[nback+1:nstat,1:nback];
MSS=W[1:nback,1:nback]+W[1:nback,nback+1:nstat]*Gamma;
PI=inv(Mcc)*(Mcs[:,1:nback]+Mcs[:,nback+1:nstat]*Gamma);
MSE=[zeros(nbend,nshoc);Matrix{Float64}(I, nshoc, nshoc)];


2×1 Array{Float64,2}:
 0.0
 1.0

In [28]:
S  = zeros(nback,long+init);
S[:,1]= MSE*e[1];
for i = 2:long+init;
   S[:,i]= MSS*S[:,i-1]+MSE*e[i];
end;

In [29]:
lb = Gamma*S;
lb = lss*exp.(lb);
lbv= vec(lb);
k  = log(kss).+S[1,:];
kv = vec(k);
ek = exp.(kv);
a  = S[2,:];
av = vec(a);
ea = exp.(av);
T  = init+1:init+long-1;
T1 = init+2:init+long;

In [30]:
XX  = [ones(long-1,1) kv[T] av[T] kv[T].*kv[T] av[T].*av[T] kv[T].*av[T]];
yy  = log.(beta*lb[T1].*(alpha*ea[T1].*ek[T1].^(alpha-1).+(1-delta)));
b0 = \(XX,yy)

6-element Array{Float64,1}:
  1.3617769676472322
 -2.434807231673646
  1.430127475094453
  0.9842358638936254
  1.3768101940834414
 -2.017018285891531

In [31]:
#Main Loop
iter=1;
crit=1;
while crit>tol
k  = zeros(slong+1,1);
lb = zeros(slong,1);
X  = zeros(slong,length(b0));
k[1]= kss;
for i= 1:slong;
  X[i,:]= [1 log.(k[i]) a[i] log(k[i])*log(k[i]) a[i]*a[i] log(k[i])*a[i]];
  lb[i]  = exp.(X[i,:]'*b0);
  k[i+1] = exp.(a[i])*k[i]^alpha+(1-delta)*k[i]-lb[i]^(-1/sigma);
end
y    = beta*lb[T1].*(alpha*exp.(a[T1]).*k[T1].^(alpha-1).+(1-delta));
bt   = X[T,:]\log.(y);
b    = copy(gam*bt+(1-gam)*b0);
crit = maximum(abs.(b-b0));
b0   = copy(b);
println("solution",b0)
iter=iter+1;
end;


solution[1.6396438277130894, -3.0070132949931625, 1.6030740449629552, 1.278463694436235, 1.4665520379608357, -2.1925633064315004]
solution[1.7681914963023138, -3.2710536423883734, 1.5384404609561018, 1.413793770696896, 1.475459422353395, -2.1241386188030607]
solution[1.8391537349375842, -3.416247698214573, 1.4473539592545592, 1.4878677368383133, 1.4937628812015558, -2.029101763386912]
solution[1.8833832791284917, -3.506356619290316, 1.372406723043625, 1.5336144990473626, 1.5193930382151912, -1.9511971860396473]
solution[1.913506682433873, -3.5675034408702393, 1.3182872994737171, 1.564534494201168, 1.5446319903244146, -1.8951248088662944]
solution[1.9354704392199458, -3.6119851300768286, 1.2814729237736109, 1.586975333064135, 1.5659699165047427, -1.857135869432492]
solution[1.9522804203455268, -3.646000079572809, 1.2572977660424287, 1.6041244925677507, 1.5827111748753504, -1.8323235029306737]
solution[1.9655268472263898, -3.6728090163225677, 1.2417427394544924, 1.6176473465647125, 1.595

In [32]:
#Pkg.add("Plots")
#Pkg.add("PlotlyJS")
#using PlotlyJS
#using Plots
plotly() # Choose the Plotly.jl backend for web interactivity
#plot(k',c',linewidth=1,label="Consumption",title="Consumption vs capital stock")
scatter(k[T],k[T1],label="Capital Stock",title="k(t+1) vs. k(t)")

#Plots.plotlyjs()   # specify backend

#scatter!(k[T],k[T1], color=:blue,  label="k")

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots C:\Users\PETRE\.julia\packages\Plots\5srrj\src\backends.jl:371


<!DOCTYPE html>
 
 
 Plots.jl